## Imports and setup

In [54]:
from openai import OpenAI

from tkinter import * 
from tkinter import ttk

import sqlalchemy
import os
from tabulate import tabulate

from dotenv import load_dotenv
import os 

from sqlalchemy import create_engine, text,Column, Integer, String, Float, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship


In [ ]:

load_dotenv()

oracle_connection_string = os.getenv('ORACLE_CONNECTION_STRING')

if oracle_connection_string:
    try:
        engine = create_engine(oracle_connection_string)
        connection = engine.connect()
        print("Connected to Oracle database successfully!")
        connection.close()
    except Exception as e:
        print("Error:", e)
else:
    print("Error: ORACLE_CONNECTION_STRING environment variable is not set.")
 
engine = create_engine(oracle_connection_string)
connection = engine.connect()


## LLM Helper Functions

In [57]:
with open('primer.txt', 'r') as file:
    primer = file.read()


primobj = {'role':'system','content':primer}

In [58]:
client = OpenAI()
def convCompletion(completion):
    return {'role':'assistant','content':completion.choices[0].message.content}
def convQuery(query):
    return {'role': 'user','content':query}

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     primobj,
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."},
#   ]
# )

def doConv(t=3):
    msgs = [primobj]
    for i in range(t):
        a = input('Enter query')
        query = convQuery(a)
        msgs.append(query) 
        completion = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=msgs
        )
        print('user:',a)
        print('bot:',completion.choices[0].message.content)
        msgs.append(convCompletion(completion))


def sendUserQuery(msg):
    query = convQuery(msg)
    msgs = [primobj, query]
    msgs.append(query) 
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=msgs
    )
    return completion.choices[0].message.content
    
def executeQuery(msg):
    print('user:',msg)
    print('bot:',sendUserQuery(msg)) 
# doConv()
        
# executeQuery('find cheapest product at store name honey farms')
# print(completion.choices[0].message.content)

## UI Code

In [46]:
def guiDoQuery(*args):
    res = sendUserQuery(userquery.get())
    print(res)
    botresponse.set(res)

def executeSQL(*args):
    try:
        runquery = connection.execute(text(sql.get()))
        result = runquery.fetchall()
        cols = runquery.keys()
        runquery.close()

        # print(type(cols))
        table = [list(cols)]+result
        temp = tabulate(table,headers='firstrow',tablefmt='fancy_grid')

        sqlresponse.set(temp)
    except Exception as e:
        sqlresponse.set(f'error: {e}')
        


In [53]:


root = Tk()
root.title("SQL LLM")



mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

userquery = StringVar()
userquery_entry = ttk.Entry(mainframe, width=10, textvariable=userquery)
ttk.Label(mainframe, text='Enter query:').grid(column=1,row=1,sticky=(W))
userquery_entry.grid(column=2, row=1, sticky=(W, E))
ttk.Button(mainframe, text='Execute Query',command=guiDoQuery).grid(row=1,column=3)

botresponse = StringVar()
botresponse.set('hi')

sql = StringVar() 
sql_entry = ttk.Entry(mainframe,width=20, textvariable=sql).grid(column=1,row=5)



sqlresponse = StringVar()
sqlresponse.set('Sample SqL response')
responselabel = ttk.Label(mainframe,padding=('5 5 5 5'),relief=SOLID,borderwidth=1,textvariable=sqlresponse).grid(column=2,row=5,rowspan=2,columnspan=10,sticky=W)

ttk.Label(mainframe,text='Bot Response').grid(column=10,row=1)
ttk.Label(mainframe, textvariable=botresponse).grid(column=11, row=1, sticky=(W, E))
# ttk.Label(mainframe, textvariable=botresponse).grid(column=3,row=2)
# ttk.Button(mainframe, text="Calculate", command=calculate).grid(column=3, row=3, sticky=W)


ttk.Label(mainframe,text='SQL To Execute:').grid(column=1,row=3,sticky=W)
ttk.Button(mainframe,text='Execute SQL',command=executeSQL).grid(column=2,row=3,sticky=W)
for child in mainframe.winfo_children(): 
    child.grid_configure(padx=5, pady=5)

userquery_entry.focus()



testquery = [ ['name','id','test'],['sdf',3,'ewr'],['sdf',3,'ewr'],['sdf',3,'ewr'],]
sqlresponse.set(tabulate(testquery,headers='firstrow',tablefmt='psql'))
root.mainloop()

## Testing SQL

In [6]:

load_dotenv()

oracle_connection_string = os.getenv('ORACLE_CONNECTION_STRING')

if oracle_connection_string:
    try:
        engine = create_engine(oracle_connection_string)
        connection = engine.connect()
        print("Connected to Oracle database successfully!")
        connection.close()
    except Exception as e:
        print("Error:", e)
else:
    print("Error: ORACLE_CONNECTION_STRING environment variable is not set.")
 
engine = create_engine(oracle_connection_string)
connection = engine.connect()

# statement = text("SELECT * FROM members" )


Connected to Oracle database successfully!


In [28]:
runquery = connection.execute(text('SELECT * FROM MEMBERS'))
result = runquery.fetchall()
cols = runquery.keys()
runquery.close()

# print(type(cols))
table = [list(cols)]+result

print(tabulate(table,headers='firstrow',tablefmt='fancy_grid'))

╒═══════╤════════════╤═════════════════╤═══════════════╕
│   mid │ name       │ address         │   phonenumber │
╞═══════╪════════════╪═════════════════╪═══════════════╡
│     1 │ John Smith │ 123 Main Street │             1 │
├───────┼────────────┼─────────────────┼───────────────┤
│     2 │ Alice Lee  │ 456 Elm Avenue  │             1 │
╘═══════╧════════════╧═════════════════╧═══════════════╛


In [13]:
print(list(cols))


['mid', 'name', 'address', 'phonenumber']


## Testing Suite

In [61]:
file = open('userqueries.txt','r')

restable=[['User Query','Generated SQL','Is Correct']]
userqueries = [l for l in file]

for i in range(len(userqueries)):
    r = sendUserQuery(userqueries[i])
    restable.append([userqueries[i],r,False])


print(tabulate(restable,headers='firstrow',tablefmt='fancy_grid'))
    



╒══════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════╤══════════════╕
│ User Query                                           │ Generated SQL                                                        │ Is Correct   │
╞══════════════════════════════════════════════════════╪══════════════════════════════════════════════════════════════════════╪══════════════╡
│ Get address of member 3                              │ SELECT address                                                       │ False        │
│                                                      │ FROM Members                                                         │              │
│                                                      │ WHERE mid = 3;                                                       │              │
├──────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────┼──────────────┤

In [ ]:
table = [['Modle','']]